In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [ ]:
system_prompt = """ 
너는 나랑 말은 릴레이 소설을 쓸거야. 서로 턴마다 한 문장씩 말하자!
어떤 장르의 소설을 쓸지는 곧바로 말해줄테니까 먼저 말하지 말고.
하지만 어떤 장르던 너는 노벨 문학상을 탈 만큼 그 장르에서 최고의 찬사를 받는 권위자야.
명심해. 전체 스토리 흐름을 파악하고 그에 알맞게 대답을 이어가야해.
그리고 나한테 말할 때는 반말해. 소설 쓸 때는 문맥에 맞게 알아서 하고.
"""

messages=[
    {"role": "system", "content": system_prompt}
]

In [17]:
from openai import OpenAI


def relay_novel(messages):
    client = OpenAI()

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages= messages,
        temperature=0.7,
        max_tokens=1028,
        top_p=0.8
    )
    
    return response.choices[0].message.content

In [18]:
def trim_messages(messages, max_turns=20):
    """
    대화가 너무 길어져서 토큰 한도를 초과하지 않도록,
    시스템 메시지(규칙) + 최근 max_turns(턴 수)의 user/assistant 대화만 남기고
    오래된 대화는 잘라내는 함수.
    """
    # messages 리스트가 비었거나, 첫 번째 메시지가 system 역할이 아닐 경우 그대로 반환
    if not messages or messages[0].get("role") != "system":
        return messages  

    # 항상 유지해야 하는 system 메시지 (게임 규칙 등)
    system = messages[0:1]  

    # system 이후의 모든 user/assistant 메시지
    rest = messages[1:]      

    # max_turns 만큼의 최근 대화만 남김
    # 한 턴은 (user + assistant) 2개의 메시지이므로 *2를 해줌
    trimmed = rest[-(max_turns*2):]  

    # system + 최근 대화만 합쳐서 반환
    return system + trimmed

In [22]:
!pip install pydub simpleaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 6.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'simpleaudio' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'simpleaudio'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for simpleaudio: filename=simpleaudio-1.0.4-cp312-cp312-macosx_11_0_arm64.whl size=2036436 sha256=7b2bdd4e081ea525ff8c4ebfc29771ef39b36601e005f6606525b205ce182055
  Stored in directory: /Users/hoon/Library/Caches/pip/wheels/1d/ad/93/be934a93f1ba8922256840f1cf0e59f8a21e2539985616b64f
Successfully built simpleaudio
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pydub]


In [ ]:
while True:
    user_line = input("나: ").strip()
    if user_line == "그만":
        print("소설 완성!")
        messages = messages[0:1]
        break

    # Add your one-sentence input
    messages.append({"role": "user", "content": user_line})
    print("나:", user_line)

    # Ask model for the next one sentence
    reply = relay_novel(messages)
    print("GPT:", reply)

    # Append assistant reply & trim to keep context light
    messages.append({"role": "assistant", "content": reply})
    messages = trim_messages(messages, max_turns=20)

나: 코미디 장르로 해보자. 자 너부터 시작!
GPT: 한 마을에 이상한 습관을 가진 사람들이 살고 있었다.
나: 무슨 습관이냐면, 모두들 10초마다 윙크를 한 번씩 해야하는 것이었다.
GPT: 이 윙크를 하지 않으면 마을 사람들은 즉시 웃음이 터져 나오는 이상한 벌을 받았다.
나: 벌은 누가 내렸을까? 마을의 한 남자가 의문을 품었다. "도대체 누가 우리에게 이딴 짓을 한거지"
GPT: 그 남자는 마을 회관에 모인 사람들에게 물었지만, 모두들 윙크를 하며 대답하지 못했다.
소설 완성!


In [6]:
import io
from openai import OpenAI
from pydub import AudioSegment
from pydub.playback import play

audio = AudioSegment.from_file("temp_tts.mp3", format="mp3")
play(audio)

client = OpenAI()

text = "지금부터 진짜 재밌는 이야기 하나 말해줄게."

with client.audio.speech.with_streaming_response.create(
    model="tts-1",
    voice="nova",
    input=text
) as response:
    response.stream_to_file("temp_tts.mp3")  # 바로 파일로 저장

# 저장된 파일 재생
audio = AudioSegment.from_file("temp_tts.mp3", format="mp3")
play(audio)

: 